In [ ]:
import glob
import math
import re
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder

# 空のDataFrameを定義
train = pd.DataFrame()
test = pd.DataFrame()
test_all = pd.DataFrame()
#csv読み込み
for csv_file in glob.glob('../data/train/train//*.csv'):
    train_data = pd.read_csv(csv_file)
    # DataFrameを連結する
    train = pd.concat([train, train_data])

for csv_file in glob.glob('../data/test.csv'):
    test_data = pd.read_csv(csv_file)
    # DataFrameを連結する
    test = pd.concat([test, test_data])
    test_all = pd.concat([test_all, test_data])
#不要な列を削除
train.drop(columns=['ID', '種類','地域', '面積（㎡）','市区町村コード', '都道府県名', '市区町村名',
 '土地の形状', '地区名', '用途', '間口', '延床面積（㎡）','前面道路：方位', '前面道路：種類', '都市計画',
  '前面道路：幅員（ｍ）', '取引時点', '取引の事情等', '今後の利用目的'], inplace=True)

test.drop(columns=['ID', '種類','地域', '面積（㎡）','市区町村コード', '都道府県名', '市区町村名',
 '土地の形状', '地区名', '用途', '間口', '延床面積（㎡）','前面道路：方位', '前面道路：種類', '都市計画',
  '前面道路：幅員（ｍ）', '取引時点', '取引の事情等', '今後の利用目的'], inplace=True)
#欠損値処理　リストワイズ法
train.dropna(inplace=True)

#順序エンコーディング　ラベルエンコーディング
le = LabelEncoder()
train['最寄駅：名称'] = le.fit_transform(train['最寄駅：名称'].values)
train['建物の構造'] = le.fit_transform(train['建物の構造'].values)
train['間取り'] = le.fit_transform(train['間取り'].values)
train['最寄駅：距離（分）'] = le.fit_transform(train['最寄駅：距離（分）'].values)

test['最寄駅：名称'] = le.fit_transform(test['最寄駅：名称'].values)
test['建物の構造'] = le.fit_transform(test['建物の構造'].values)
test['間取り'] = le.fit_transform(test['間取り'].values)
test['最寄駅：距離（分）'] = le.fit_transform(test['最寄駅：距離（分）'].values)
#One-Hotエンコーディング 最寄り駅 間取り　建物の構造　改装
train= pd.get_dummies(train, columns=['改装'])
test= pd.get_dummies(test, columns=['改装'])

#最寄り駅までの時間を数値に変換
#replaceDic = {}
#with open("ConvertTime.txt") as file:
#    for line in file:
#        (k, v) = line.split()
#        replaceDic[k] = int(v)
#train['最寄駅：距離（分）'] = train['最寄駅：距離（分）'].replace(replaceDic)

#和暦から西暦変換
convYear = {}
with open("ConvertYear.txt") as file:
    for line in file:
        (k, v) = line.split()
        convYear[k] = int(v)
train['建築年'] = train['建築年'].replace(convYear)

onvYear = {}
with open("ConvertYear.txt") as file:
    for line in file:
        (k, v) = line.split()
        onvYear[k] = int(v)
test['建築年'] = test['建築年'].replace(convYear)

In [ ]:
#モデル構築
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

FOLD = 5
NUM_ROUND = 100
VERBOSE_EVAL = -1

X_train = train.drop(['取引価格（総額）_log'], axis=1)
Y_train = train['取引価格（総額）_log']

#３分割交差検証を指定し、インスタンス化
kf = KFold(n_splits=FOLD, shuffle=True, random_state=42)

#スコアとモデルを格納するリスト
score_list = []
models = []

for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, Y_train)):
    print(f'fold{fold_+ 1}start')
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = Y_train.iloc[train_index]
    valid_y = Y_train.iloc[valid_index]

    #lab.Datasetを使って、trainとvalidを作っておく
    lgb_train = lgb.Dataset(train_x, train_y, categorical_feature=['最寄駅：名称', '建物の構造', '間取り'])
    lgb_valid = lgb.Dataset(valid_x, valid_y, categorical_feature=['最寄駅：名称', '建物の構造', '間取り'])

    #パラメータを定義
    lgbm_params = {'objective':'regression',
                   'verbose': -1,}

    #lgb.trainで学習
    gbm = lgb.train(params=lgbm_params,
                    train_set=lgb_train,
                    valid_sets=[lgb_train, lgb_valid],
                    num_boost_round = NUM_ROUND,
                    verbose_eval = VERBOSE_EVAL) #学習の状況を表示しない
    #off = (gbm.predict(valid_x) > 0.5).astype(int)
    y_valid_pred = gbm.predict(valid_x)
    score = mean_absolute_error(valid_y, y_valid_pred)
    print(f'fold{fold_+1} MAE:{score}')
    score_list.append(score)
    models.append(gbm)#学習が終わったらモデルをリストに入れておく
    
cv_score = np.mean(score_list)
print(f'CV score: {cv_score}')

In [ ]:
#テストデータを予測
test_pred = np.zeros((len(test), 5))

sample_submisson = pd.DataFrame(test_all['ID'])

for fold_, gbm in enumerate(models):
    pred_ = gbm.predict(test)
    test_pred[:, fold_] = pred_

sample_submisson['取引価格（総額）_log'] = test_pred[:,0]
sample_submisson.to_csv('submisson.csv', index=False)
